In [ ]:
#!/usr/bin/env python3
import rospy
from geometry_msgs.msg import Twist
from std_msgs.msg import Int16, Int16MultiArray
from sensor_msgs.msg import Imu
from sensor_msgs.msg import Range
import math
from tf.transformations import euler_from_quaternion

forward_distance = right_distance = left_distance = yaw = 0
angular_speed = 0.3

def ultrasonicf(msg):
    global forward_distance

    forward_distance = msg.range * 100
    rospy.loginfo("Forward: %d ",forward_distance)
def ultrasonicr(msg):
    global right_distance

    right_distance = msg.range * 100
    rospy.loginfo("right: %d ",right_distance)
def ultrasonicl(msg):
    global left_distance

    left_distance = msg.range * 100
    rospy.loginfo("left: %d ",left_distance)


def yaw_callback(msg):
    global yaw
    # print("received imu data")
    q = msg.orientation
    quaternion = [q.x, q.y, q.z, q.w]
    roll, pitch, yaw = euler_from_quaternion(quaternion)
    # print("Yaw (rotation about Z):", yaw)

def listener():
    rospy.init_node('robot')
    rospy.Subscriber('ultrasonic/front', Range, ultrasonicf)
    rospy.Subscriber('ultrasonic/right', Range, ultrasonicr)
    rospy.Subscriber('ultrasonic/left', Range, ultrasonicl)
    rospy.Subscriber('/imu', Imu, yaw_callback)

def normalize_angle(angle):
    """Wrap angle to [-180, 180] degrees."""
    return math.atan2(math.sin(angle), math.cos(angle))


def movement():
    cmd = Twist()

    # safety check
    if forward_distance == 0 or left_distance == 0 or right_distance == 0:
        cmd.linear.x = 0
        cmd.angular.z = 0
        pub.publish(cmd)
        return

    # If obstacle ahead
    if forward_distance < 50:
        # if left_distance > right_distance:
            turn_angle(math.pi/2, cmd)   # turn left
        # else:
            # turn_angle(-90, cmd)  # turn right
    else:
        # Move forward slowly if path is clear
        cmd.linear.x = min(0.2, forward_distance * 0.01)  # proportional, but capped
        cmd.angular.z = 0
        pub.publish(cmd)


def turn_angle(rotation_angle, cmd):
    global yaw
    """Turn the robot by a relative angle (radians)."""
    start_yaw = yaw
    target_yaw = normalize_angle(start_yaw + rotation_angle)
    rate = rospy.Rate(10)

    error = normalize_angle(target_yaw - yaw)
    while not rospy.is_shutdown() and abs(error) > 0.1:
        cmd.angular.z = math.copysign(angular_speed, error)
        cmd.linear.x = 0
        pub.publish(cmd)
        rate.sleep()
        error = normalize_angle(target_yaw - yaw)  # update inside loop



def main():
    global pub
    pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
    
    listener()

    while not rospy.is_shutdown():
        movement()
        

if __name__ == "__main__":
    main()

# roslaunch turtlebot3_gazebo turtlebot3_house.launch
# roslaunch turtlebot3_gazebo turtlebot3_stage_4.launch
# :~/catkin_ws/src/turtlebot3/turtlebot3_description/urdf

In [ ]:
import threading
import time



class PID():

    def __init__(self, set_point, kp_set_point = 1.0, kd_set_point = 1.0, ki_set_point = 1.0, max_limit_set_point = 1.0, min_limit_set_point = 0.0) -> None:
        self.set_point = set_point

        self.max_set_point = max_limit_set_point
        self.min_set_point = min_limit_set_point
        self.kp_set_point = kp_set_point
        self.kd_set_point = kd_set_point
        self.ki_set_point = ki_set_point
        self.i_set_point = 0.0
        self.max_i_set_point = max_limit_set_point / ki_set_point
        self.min_i_set_point = min_limit_set_point / ki_set_point
        self.set_point_feedback = 0
        self.last_error_set_point = 0

        self.t1 = time.time()

        self.running = True
        self.time = 0.5
        threading.Thread(target=self.update_set_point, daemon=True).start()

    def __str__(self) -> str:
        return f"PID(set_point = {self.set_point}, time = {self.time}, {' '*99}\n" \
        f"kp_set_point = {self.kp_set_point}, kd_set_point = {self.kd_set_point}, ki_set_point = {self.ki_set_point},\n" \
        f"max = {self.max_set_point}, min = {self.min_set_point})\n"
    
    def __setattr__(self, name: str, value) -> None:
        if name in ["kp_set_point", "kd_set_point", "ki_set_point"]:
            if value <= 0 or value >= 1: 
                print("\rInvalid value", "   " * 10, end="\r")
                time.sleep(1)
                return
        self.__dict__[name] = value
    
    def update_set_point(self, set_point_feedback = None):
        time.sleep(self.time)
        if set_point_feedback is None: set_point_feedback = self.set_point_feedback
        error_set_point = self.set_point - set_point_feedback
        dt = time.time() - self.t1
        if dt < 0.1: dt = 0.1
        p = error_set_point * self.kp_set_point
        i_set_point = self.i_set_point + error_set_point * self.ki_set_point * dt 
        i_set_point = max(self.min_i_set_point, min(self.max_i_set_point, i_set_point))
        d = self.kd_set_point * (error_set_point - self.last_error_set_point) / dt
        out = p + i_set_point + d
        self.set_point_feedback = min(self.max_set_point, max(self.min_set_point, out))
        print(f"error_set_point = {error_set_point:.2f}, p = {p:.2f}, i = {i_set_point:.2f}, d = {d:.2f}, out = {out:.2f}, set_point_feedback = {self.set_point_feedback:.2f}", "   " * 10, end="\r")
        self.last_error_set_point = error_set_point
        self.i_set_point = i_set_point
        self.t1 = time.time()
        return self.set_point_feedback
    
    def move(self, set_point):
        self.set_point = set_point

    def rotate(self, angel):
        self.angel = angel

    def printer(self):
        self.update_set_point()
        time.sleep(self.time)

In [ ]:
#!/usr/bin/env python3
import rospy
from geometry_msgs.msg import Twist
from std_msgs.msg import Int16, Int16MultiArray
from sensor_msgs.msg import Imu
from sensor_msgs.msg import Range
import math
from tf.transformations import euler_from_quaternion

forward_distance = right_distance = left_distance = yaw = angel_on_the_horizontal = 0
angular_speed = 0.3

def ultrasonicf(msg):
    global forward_distance
    forward_distance = msg.range * 100
    rospy.loginfo("Forward: %d ",forward_distance)

def ultrasonicr(msg):
    global right_distance
    right_distance = msg.range * 100
    rospy.loginfo("right: %d ",right_distance)

def ultrasonicl(msg):
    global left_distance
    left_distance = msg.range * 100
    rospy.loginfo("left: %d ",left_distance)

def yaw_callback(msg):
    global yaw
    # print("received imu data")
    q = msg.orientation
    quaternion = [q.x, q.y, q.z, q.w]
    roll, pitch, yaw = euler_from_quaternion(quaternion)
    # print("Yaw (rotation about Z):", yaw)

def listener():
    rospy.init_node('robot')
    rospy.Subscriber('ultrasonic/front', Range, ultrasonicf)
    rospy.Subscriber('ultrasonic/right', Range, ultrasonicr)
    rospy.Subscriber('ultrasonic/left', Range, ultrasonicl)
    rospy.Subscriber('/imu', Imu, yaw_callback)

def normalize_angle(angle):
    """Wrap angle to [-180, 180] degrees."""
    return math.atan2(math.sin(angle), math.cos(angle))

def movement():
    cmd = Twist()

    # safety check
    if forward_distance == 0 or left_distance == 0 or right_distance == 0:
        cmd.linear.x = 0
        cmd.angular.z = 0
        pub.publish(cmd)
        return

    # If obstacle ahead
    if forward_distance < 40:
        turn_angle(math.pi/2, cmd) # turn left for now, untill we have cv readings

    else:
        cmd.linear.x = min(0.1, forward_distance * 0.01)
        cmd.angular.z = 0
        pub.publish(cmd)


def turn_angle(rotation_angle, cmd):
    """Turn the robot by a relative angle (radians)."""
    global target_yaw
    start_yaw = yaw
    target_yaw = normalize_angle(start_yaw + rotation_angle)
    rate = rospy.Rate(10)

    error = normalize_angle(target_yaw - yaw)
    while not rospy.is_shutdown() and abs(error) > 0.1:
        cmd.angular.z = math.copysign(angular_speed, error)
        cmd.linear.x = 0
        pub.publish(cmd)
        rate.sleep()
        error = normalize_angle(target_yaw - yaw)  # update inside loop

def angel_using_usltrasonic():
    global angel_on_the_horizontal
    width = left_distance + right_distance + 33
    angel_on_the_horizontal = math.atan2(44.5, width)
    return angel_on_the_horizontal

def main():
    global pub
    pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
    
    listener()

    while not rospy.is_shutdown():
        movement()
        

if __name__ == "__main__":
    main()

# roslaunch turtlebot3_gazebo turtlebot3_house.launch
# roslaunch turtlebot3_gazebo turtlebot3_stage_4.launch
# :~/catkin_ws/src/turtlebot3/turtlebot3_description/urdf

In [ ]:
def turn_angle(rotation_angle, cmd):
    """Turn the robot by a relative angle (radians)."""
    global target_yaw
    start_yaw = yaw
    target_yaw = normalize_angle(start_yaw + rotation_angle)
    rate = rospy.Rate(10)

    error = normalize_angle(target_yaw - yaw)
    while not rospy.is_shutdown() and abs(error) > 0.01:
        if abs(error) > 1: cmd.angular.z = math.copysign(0.8, error)
        elif abs(error) > 0.5: cmd.angular.z = math.copysign(0.4, error)
        elif abs(error) > 0.25: cmd.angular.z = math.copysign(0.2, error)
        elif abs(error) > 0.1: cmd.angular.z = math.copysign(0.1, error)
        elif abs(error) > 0.01: cmd.angular.z = math.copysign(0.05, error)
        cmd.linear.x = 0
        pub.publish(cmd)
        rate.sleep()
        error = normalize_angle(target_yaw - yaw)  # update inside loop

In [ ]:
import math


def angel_using_usltrasonic(left_distance, right_distance):
    global angel_on_the_horizontal
    width = left_distance + right_distance + 33
    angel_on_the_horizontal = math.degrees(math.acos(44.5/width))
    return angel_on_the_horizontal

angel_on_the_horizontal = 0
print(angel_using_usltrasonic(5.75, 5.75))

In [ ]:
%pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.1/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.1/index.html
Note: you may need to restart the kernel to use updated packages.
done2


ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [1]:
from segment_anything import sam_model_registry, SamPredictor
import torch
import cv2

sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h.pth")
predictor = SamPredictor(sam)


FileNotFoundError: [Errno 2] No such file or directory: 'sam_vit_h.pth'

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\ahmos\appdata\local\temp\pip-req-build-pswxyatn
  Resolved https://github.com/facebookresearch/detectron2.git to commit a1ce2f956a1d2212ad672e3c47d53405c2fe4312
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\ahmos\AppData\Local\Temp\pip-req-build-pswxyatn'
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "c:\Users\ahmos\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\ahmos\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\ahmos\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143,